In [1]:
%matplotlib inline

In [2]:
import os
import torch

In [3]:
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
print(device)

cuda:5


In [4]:
%load_ext autoreload
%autoreload 2

# load data

In [180]:
from transformer_translation.dataset import TagReportDataset
from torch.utils.data import DataLoader

In [181]:
data_path = r"/home/alex/data/nlp/agmir/transf_processed_data"
#data_path = 'transformer_translation/data/processed'

In [182]:
num_tokens = 2000
max_seq_length = 96
dataset = TagReportDataset(
    os.path.join(data_path, 'tags/set_raw.pkl')
    ,os.path.join(data_path, 'reports/set.pkl')
    ,num_tokens
    ,max_seq_length)
loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)

In [164]:
(src, src_key_padding_mask, tgt, tgt_key_padding_mask) = next(iter(loader))

In [165]:
src.shape, src_key_padding_mask.shape

(torch.Size([1, 52, 587]), torch.Size([1, 52, 1]))

In [166]:
tgt.shape, tgt_key_padding_mask.shape

(torch.Size([1, 52, 40]), torch.Size([1, 52, 40]))

In [167]:
            src, src_key_padding_mask  = src.to(device), src_key_padding_mask[0].to(device)
            tgt, tgt_key_padding_mask = tgt[0].to(device), tgt_key_padding_mask[0].to(device)

            memory_key_padding_mask = src_key_padding_mask.clone()
            tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = gen_nopeek_mask(tgt_inp.shape[1]).to(device, dtype=src.dtype)

In [169]:
src[0].shape, src.shape, tgt_inp.shape, src_key_padding_mask.shape, tgt_key_padding_mask[:, :-1].shape, memory_key_padding_mask.shape, tgt_mask.shape

(torch.Size([52, 587]),
 torch.Size([1, 52, 587]),
 torch.Size([52, 39]),
 torch.Size([52, 1]),
 torch.Size([52, 39]),
 torch.Size([52, 1]),
 torch.Size([39, 39]))

In [170]:
src[0].dtype, src.dtype, tgt_inp.dtype, src_key_padding_mask.dtype, tgt_key_padding_mask[:, :-1].dtype, memory_key_padding_mask.dtype, tgt_mask.dtype

(torch.int64,
 torch.int64,
 torch.int64,
 torch.bool,
 torch.bool,
 torch.bool,
 torch.int64)

In [179]:
outputs = model(src, tgt_inp, src_key_padding_mask, tgt_key_padding_mask[:, :-1], memory_key_padding_mask, tgt_mask)

# train

In [183]:
from transformer_translation.model import LanguageTransformer, ReportTransformer
from transformer_translation.translate_sentence import gen_nopeek_mask

In [184]:
vocab_size = 10000 + 4#1952#
nhead = 8
d_model = 587 - (587 % nhead) + nhead
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048
pos_dropout = 0.1
trans_dropout = 0.1

model = ReportTransformer(
    vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward,
    max_seq_length, pos_dropout, trans_dropout
).to(device)

In [185]:
    from transformer_translation.Optim import ScheduledOptim
    import torch.nn as nn
    from torch.optim import Adam
    
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_normal_(p)

    n_warmup_steps = 4000
    optim = ScheduledOptim(
        Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
        d_model, n_warmup_steps)

    criterion = nn.CrossEntropyLoss(ignore_index=0)

In [186]:
    %%time
    print_every = 15
    num_epochs = 2
    model.train()

    lowest_val = 1e9
    val_losses = []
    total_step = 0
    
    for epoch in range(num_epochs):
        
        total_loss = 0

        for step, (src, src_key_padding_mask, tgt, tgt_key_padding_mask) in enumerate(iter(loader)):
            total_step += 1

            src, src_key_padding_mask = src.to(device), src_key_padding_mask[0].to(device)
            tgt, tgt_key_padding_mask = tgt[0].to(device), tgt_key_padding_mask[0].to(device)

            memory_key_padding_mask = src_key_padding_mask.clone()
            tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = gen_nopeek_mask(tgt_inp.shape[1]).to(device)

            optim.zero_grad()
            outputs = model(src, tgt_inp, src_key_padding_mask, tgt_key_padding_mask[:, :-1], memory_key_padding_mask, tgt_mask)
            loss = criterion(rearrange(outputs, 'b t v -> (b t) v'), rearrange(tgt_out, 'b o -> (b o)'))

            loss.backward()
            optim.step_and_update_lr()

            total_loss += loss.item()
            if step % print_every == print_every - 1:
                print(f'Epoch [{epoch + 1} / {num_epochs}] \t Step [{step + 1} / {len(loader)}] \t '
                      f'Train Loss: {total_loss / print_every}')
                total_loss = 0
                #print(get_sent_from_tk(
                #    rearrange(outputs, 'b t v -> (b t) v').max(dim=1)[1].unique()
                #    ,reports_index2word))

Epoch [1 / 2] 	 Step [15 / 99] 	 Train Loss: 9.24535477956136
Epoch [1 / 2] 	 Step [30 / 99] 	 Train Loss: 9.005743090311686


RuntimeError: The size of tensor a (97) must match the size of tensor b (96) at non-singleton dimension 0

# get nat lg

In [145]:
rearrange(torch.cat((
            rearrange(src[0], 'n s -> s n')
            ,rearrange(torch.zeros([, 5]).to(device, dtype=src.dtype), 'n s -> s n')
        )), 's n -> n s').unsqueeze(0)

EinopsError:  Error while processing rearrange-reduction pattern "n s -> s n".
 Input tensor shape: torch.Size([587]). Additional info: {}.
 Expected 2 dimensions, got 1

In [160]:
src.dtype, tgt.dtype, src_key_padding_mask.dtype, tgt_key_padding_mask.dtype

(torch.int64, torch.int64, torch.bool, torch.bool)

In [15]:
from einops import rearrange

In [174]:
type(d_model),type(587)

(int, int)

In [177]:
torch.float32

torch.float32

In [16]:
import pickle
with open(os.path.join(data_path, 'tags', 'voc.pkl'), 'rb') as f:
    tags_word2index = pickle.load(f)
tags_index2word = dict(zip(tags_word2index.values(), tags_word2index.keys()))
tags_index2word[0], tags_index2word[1] = "SOS", "EOS"
    
with open(os.path.join(data_path, 'reports', 'voc.pkl'), 'rb') as f:
    reports_word2index = pickle.load(f)
reports_index2word = dict(zip(reports_word2index.values(), reports_word2index.keys()))
reports_index2word[0], reports_index2word[1] = "SOS", "EOS"

In [17]:
def get_sent_from_tk(tensor_tk, index2word):
    return [index2word[idx.item()] for idx in tensor_tk]

In [18]:
get_sent_from_tk(
    rearrange(tgt, 'b o -> (b o)')
    ,reports_index2word)

['',
 '',
 '.',
 'chest',
 '.',
 'no',
 'active',
 'disease',
 '.',
 '.',
 'left',
 'and',
 'right',
 'xxxx',
 '.',
 'bilateral',
 'degenerative',
 'joint',
 'disease',
 'left',
 'worse',
 'than',
 'right',
 '.',
 'chest',
 '.',
 'the',
 'heart',
 'and',
 'lungs',
 'have',
 'xxxx',
 'xxxx',
 'in',
 'the',
 'interval',
 '.',
 'both',
 'lungs',
 'are',
 'clear',
 'and',
 'expanded',
 '.',
 'heart',
 'and',
 'mediastinum',
 'normal',
 '.',
 'left',
 'and',
 'right',
 'xxxx',
 '.',
 'osteophytes',
 'are',
 'present',
 'at',
 'the',
 'acromioclavicular',
 'joints',
 'bilaterally',
 'and',
 'also',
 'on',
 'the',
 'humeral',
 'necks',
 '.',
 'the',
 'right',
 'glenohumeral',
 'joint',
 'is',
 'normal',
 'but',
 'the',
 'left',
 'is',
 'narrowed',
 '.',
 'no',
 'fractures',
 'or',
 'bone',
 'destruction',
 '.',
 '.',
 'SOS',
 'SOS',
 'SOS',
 'SOS',
 'SOS',
 'SOS',
 'SOS',
 'SOS',
 '',
 'no',
 'acute',
 'cardiopulmonary',
 'abnormality',
 '.',
 'crowded',
 'bronchovascular',
 'markings',
 'in'

In [71]:
get_sent_from_tk(
    rearrange(src, 'b o -> (b o)')
    ,tags_index2word)

['infiltrates',
 'cardiomegaly',
 'granuloma',
 'granuloma',
 'aorta',
 'left_ventricle',
 'atelectases',
 'atelectasis',
 'granuloma',
 'granuloma',
 'granuloma',
 'granuloma',
 'rib_fracture',
 'rib_fractures',
 'plate_like_atelectasis',
 'pulmonary_atelectasis',
 'atelectases',
 'atelectasis',
 'lymph',
 'lymph_nodes',
 'scarring',
 'scarring',
 'scolioses',
 'scoliosis',
 'tortuous_aorta',
 'aorta',
 'pneumonia',
 'pneumonia',
 'sternotomy',
 'sternotomy',
 'hiatal_hernia',
 'hiatal_hernia',
 'spinal_osteophytosis',
 'thoracic_spondylosis',
 'clip',
 'gallbladder',
 'right_lower_lobe_pneumonia',
 'pneumonia',
 'granuloma',
 'granuloma',
 'aorta',
 'aortic_diseases',
 'apical_granuloma',
 'granuloma',
 'spondylosis',
 'spondylosis',
 'spondylosis',
 'spondylosis',
 'atelectases',
 'atelectasis',
 'tortuous_aorta',
 'aorta',
 'heart_valve_prosthesis',
 'heart_valve_prosthesis_implantation',
 'granulomatous_disease',
 'thoracic_spondylosis',
 'aorta_tortuous',
 'aorta',
 'atherosclero

592

In [70]:
get_sent_from_tk(
    rearrange(outputs, 'b t v -> (b t) v').max(dim=1)[1].unique()
    ,reports_index2word)

['.']

In [75]:
592/8

74.0